# Pinyin Syllable Count Generator Working Draft Notes
This document details the process of experimentation related to the construction of the Pinyin Syllable Count Generator module. The module takes two columns of string data, one with unboken text in Chinese, the other with its pinyin counterpart. The package will evaluate the number of syllables in the chinese text based on the number of syllables it can detect in the supplied Latin characters. The results will be delivered as integers with commas separating the syllables in each individual word.

The Pandas package will be essential for the vector manipulation and storage.

In [1]:
import pandas as pd
import numpy as np
import csv
import sys

Because pinyin is a structured system with accepted combinations of "initials" (the initial sound of a syllable, e.g. the first consonant) and "finals" (the ending sound of a syllable, e.g. vowels ending with a consonant). One syllable in pinyin can be as small as one letter and as large as six letters. Therefore, detecting a syllable can occur in one of two ways:
1. A sequence of pinyin letters can be compared to a list of accepted pinyin syllables. If the sequence appears in the list, or if the sequence is part of a longer accepted term ("tia" by itself is not an accepted syllable, however "tiao" and "tian" are), then an additional letter from the evaluated word can be added to the sequence and reevaluated. If the sequence does not appear in the list and is not part of a longer accepted term, then the last character of the sequence becomes the first character of the next syllable and the syllable count is incremented.
2. A sequence of pinyin letters is evaluated against a DataFrame of initials and finals, utilizing the .loc command to spot matches if they exist or null values if they don't.

Given these possiblities two metrics need to be considered in order to settle on the proper method for the evaluation function: speed and size. Comparing a sequence to a list of terms can be faster, however the list itself may be excessively large. Evaluating sequences by their combination of initials and finals, however, may take longer but will be result in a much smaller package. The following exercises before will test each method in a smaller scale to get a general sense of the speed vs. size issue.

## Setup for small tests

An example sequence of letters will be tested with both methods.

In [2]:
# Sample sequence
smEval = 'teng'

## LIST ##
# Import list of all pinyin combinations
smList = []
with open("pinyinList.csv") as f:
    r = csv.reader(f)
    for row in r:
        smList += row
        
## NUMPY/DATAFRAME ##
# Variables for evaluation
smInitial = 't'
smFinal = 'eng'

# List of initials, finals for Numpy evaluation
initList = ['ø','b','p','m','f','d','t','n','l','z','c','s',
            'zh','ch','sh','r','j','q','x','g','k','h','y','w']
finList = ['a','ai','ao','an','ang','e','ei','en','eng','er',
           'o','ong','ou','i','ia','iao','ie','iu','ian','iang',
           'in','ing','iong','u','ua','uo','ue','uei','uai','uan','uen','uang','v']

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

# Import dataframe from CSV 
smDF = pd.read_csv('pinyinDF.csv', header=0, index_col=0, dtype={'INDEX':str})
smDF = smDF.fillna(0).astype('bool')

print('Python List:')
print(str(smList[50:55]) + '\n')
print('Numpy Array:')
print(str(smAr[0])  + '\n')
print('Pandas DataFrame:')
print(smDF.head(3))

Python List:
['bu', 'pa', 'pai', 'pao', 'pan']

Numpy Array:
[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False]

Pandas DataFrame:
      a    ai    ao    an   ang      e    ei    en   eng     er  ...      u  \
ø  True  True  True  True  True   True  True  True  True   True  ...  False   
b  True  True  True  True  True  False  True  True  True  False  ...   True   
p  True  True  True  True  True  False  True  True  True  False  ...   True   

      ua     uo     ue    uei    uai    uan    uen   uang      v  
ø  False  False  False  False  False  False  False  False  False  
b  False  False  False  False  False  False  False  False  False  
p  False  False  False  False  False  False  False  False  False  

[3 rows x 33 columns]


In [3]:
# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial),finList.index(smFinal)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal][smInitial] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial,smFinal] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial),finList.index(smFinal)] != True

Python List: 3760
1.31 µs ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1488, without initials and finals lists: 904
1.46 µs ± 51 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 2395
slice
9.32 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
7.65 µs ± 10 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.21 µs ± 40.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [130]:
def sylCount(pinyin):
    '''Count the number of Chinese characters based on inputted pinyin text.'''

    # Lists for reference
    consonant = ['b','c','ch','d','f','g','h','j','k','l','m','n',
                 'p','q','r','s','sh','t','w','x','y','z','zh']
    vowel = ['a','e','i','o','u','v']
    
    # Variables used for pinyin evaluation
    
    pinyin = pinyin.strip()
    
    pEval = ''
    initial = ''
    final = ''
    result = False
    initAr = []
    space = False
    syl = 1
    sylCount = []
    
    
    for i,n in enumerate(str.lower(pinyin)):
        
        if n == ' ':
            if not space:
                sylCount.append(syl)
                initial = ''
                final = ''
                syl = 1
            if pinyin[i+1] == ' ':
                space = True
                #print('0. ' + ' space - double space')
            else:
                #print('0. ' + ' space - new word')
                space = False
            #print('0. ' + initial + '|' + final + n + ' = space - new word')
        elif final:
            try:
                result = smAr[initList.index(initial),finList.index(final + n)]
                #print('1a. ' + initial + '|' + final + n + ' = init - trying: ' + str(result))
            except:
                result = False
                #print('1b. ' + initial + '|' + final + n + ' = fin - error')
            
            if not result and not any(i for i in finList if (final+n) in i):
                syl += 1
                if n in vowel:
                    initial = 'ø'
                    final = n
                else:
                    initial = n
                    final = ''
                result = False
                #print('2a. ' + initial + '|' + final + ' = fin - new syl = ' + str(syl))
            else:
                final += n
                #print('2b. ' + initial + '|' + final + ' = fin - more')
        elif n not in vowel:
            initial += n
            #print('3. ' + initial + ' = init - cons')
        elif not initial:
            initial = 'ø'
            final = n
            #print('4. ' + initial + '|' + final + ' = init - vowel')
        else:
            final = n
            #print('5. ' + initial + '|' + final + ' = fin - vowel')
    
    
    '''
    
    for n in pinyin:
        if (not initial) or (initial in ['c','s','z'] and n == 'h'):
            initial += n
            print('1 - ' + initial)
            
        if initial and len(initAr) == 0:
            if initial in vowel:
                initial = 'ø'
                final = n
            initAr = smAr[initList.index(initial)]
            print('2 - ' + initial)
        else:
            final += n
            print('3 - ' + final)
            try:
                initAr[finList.index(final)]
                print('4 - ' + initial + final)
            except:
                syl += 1
                initial = n
                final = ''
                initAr = []
                print('syl = ' + str(syl))
    '''
    
    sylCount.append(syl)
    
    return sylCount

In [131]:
sylCount('Hao A Shinahuseluo Chaowen')

[1, 1, 5, 2]

In [91]:
any(i for i in finList if 'aoz' in i)

False

In [45]:
smAr[initList.index('w'),finList.index('eiz')]

ValueError: 'eiz' is not in list

In [7]:
def sylCount(word):
    
    tEval = ''
    syl = 1
    
    for n in word:
        tEval = tEval + n
        if (tEval not in varList) and not any(i for i in varList if tEval in i):
            syl += 1
            tEval = n
    return syl

In [132]:
names = pd.read_csv('Chinese Names.csv')
names[names['ChiName'].str.len() > 5]

,EngName,ChiName
102,A Shinaaboshe,阿史那阿波設
104,A Shinachuoluokehan,阿史那啜羅可汗
106,A Shinahuseluo,阿史那斛瑟羅
107,A Shinaqiminkehan,阿史那啟民可汗
120,Abilule Suofeiyinga,阿畢魯勒索費英阿
123,Aboerjijite Aleqinga,阿博爾濟吉特阿勒清阿
152,Ademishimenggu,阿的彌失蒙古
153,Ademishitiemuer,阿的迷失帖木兒
156,Adouhalamanhudoupuhuayaozhu,阿都哈剌蠻護都普花咬住
170,Aerhaihesaer,阿兒孩合撒兒


In [133]:
names['syl'] = names['EngName'].apply(sylCount)

In [128]:
names['sylCount'] = names['syl'].str.split(',')

In [129]:
names

,EngName,ChiName,syl,check,sylCount
0,Aduan,阿段,2,True,NaN
1,Aji,阿稽,2,True,NaN
2,Arong,阿容,2,True,NaN
3,Aruan,阿軟,2,True,NaN
4,Awang,阿汪,2,True,NaN
5,Azui,阿最,3,False,NaN
6,Beidu,杯渡,2,True,NaN
7,Biyu,碧玉,2,True,NaN
8,Chaozong,朝宗,2,True,NaN
9,Chisong,赤松,2,True,NaN
